In [67]:
import SPiiPlusPython as sp 

In [68]:
# Open serial connection
hc = sp.OpenCommSerial(1, 115200)

In [74]:
# Running buffer 1 at the startup level for homing
sp.RunBuffer(hc, 1, "STARTUP", sp.SYNCHRONOUS, True)

In [70]:
# Enable X-axis
sp.Enable(hc, 0, sp.SYNCHRONOUS, True)
# Enable Y-axis
sp.Enable(hc, 1, sp.SYNCHRONOUS, True)

In [ ]:
# Moving axis to a point at a set velocity
sp.ExtToPoint(
    hc,
    sp.MotionFlags.ACSC_AMF_VELOCITY,
    sp.Axis.ACSC_AXIS_0,
    # millimeters
    0,
    # millimeters per second
    velocity=15,
    wait=sp.SYNCHRONOUS,
    failure_check=True
)

In [106]:
# Moving multiple axes to a single point at specified velocity
sp.ExtToPointM(
    hc,
    sp.MotionFlags.ACSC_AMF_VELOCITY | sp.MotionFlags.ACSC_AMF_ENDVELOCITY,
    # Need the -1 at the end to mark the end of the axis array
    [0,1,-1],
    # X, Y points
    point=[-10,-10],
    velocity=50,
    endVelocity= 0,
    failure_check=True
)

In [ ]:
# Single axis job with specified velocity (Could set up with timing or external trigger?)
sp.Jog(
    hc, 
    sp.MotionFlags.ACSC_AMF_VELOCITY, 
    sp.Axis.ACSC_AXIS_1,
    # Direction and velocity (Positive or negative value)
    10,
    failure_check=True
    )

In [ ]:
# Single axis, multiple points

sp.MultiPoint(
    hc,
    sp.MotionFlags.ACSC_AMF_VELOCITY,
    sp.Axis.ACSC_AXIS_1,
    1,
    failure_check=True,
)

# ExtAddPoint allows you to specify the values for the flags in the method, in this case, velocity
sp.ExtAddPoint(hc, sp.Axis.ACSC_AXIS_1, 10, 25)
sp.ExtAddPoint(hc, sp.Axis.ACSC_AXIS_1, -10, 15)
sp.ExtAddPoint(hc, sp.Axis.ACSC_AXIS_1, 15, 15)
sp.ExtAddPoint(hc, sp.Axis.ACSC_AXIS_1, 0, 15)


sp.EndSequence(hc, sp.Axis.ACSC_AXIS_1)


In [51]:
# Multi-axis, multiple points (Drawing a square)
axes = [0,1,-1]
sp.MultiPointM(
    hc,
    sp.MotionFlags.ACSC_AMF_VELOCITY,
    axes,
    1,
    failure_check=True
)

sp.ExtAddPointM(hc, axes, (0,0), 25)
for i in range(10):
    sp.ExtAddPointM(hc, axes, (-10,10), 25)
    sp.ExtAddPointM(hc, axes, (10,10), 25)
    sp.ExtAddPointM(hc, axes, (10,-10), 25)
    sp.ExtAddPointM(hc, axes, (-10,-10), 25)
sp.ExtAddPointM(hc, axes, (0,0), 25)
sp.EndSequenceM(hc, axes) # end of multipoint motion

In [20]:
# Single axis spline
sp.Spline(hc, sp.MotionFlags.ACSC_AMF_CUBIC, 0, 1000, sp.SYNCHRONOUS, True)
for i in range(5):
    sp.AddPVPoint(hc, 0, i*1.5, 25, sp.SYNCHRONOUS, True)
sp.EndSequence(hc, 0, sp.SYNCHRONOUS, True)

In [ ]:
# Multi-axis spline (Drawing a circle)

import math
axes = (0, 1, -1)

sp.SplineM(
    hc,
    sp.MotionFlags.ACSC_AMF_CUBIC,
    axes,
    1000,
    failure_check=True
)

# Drawing a circle
radius = 10
num_steps = 5
angular_speed = 2 * math.pi / (num_steps * 1.0)

for i in range(num_steps):
    angle = (2 * math.pi / num_steps) * i
    x = radius * math.cos(angle)
    y = radius * math.sin(angle)
    vx = -radius * math.sin(angle) * angular_speed
    vy = radius * math.cos(angle) * angular_speed
    sp.AddPVPointM(hc, axes, (x, y), (vx, vy))

# PV = Position + Velocity
# PVT = Position + Velocity + Time -> Can use for laser synchronization
sp.AddPVPointM(hc, axes, (radius, 0), (0, angular_speed * radius))
sp.EndSequenceM(hc, axes)


In [ ]:
# For converting DXF to something that can be used here:
# 
# Need to convert the DXF into segments:
#   1. Line segments
#   2. Arc segments
# 
# Non-uniform rational basis spline 
# could be better than cubic interpolation
#
#
#
#

In [71]:
import ezdxf

outline = ezdxf.readfile(r'C:\Users\max\Desktop\NanoStride\test_files\Nov2025design_nanoclip_outline_v4.dxf')
msp = outline.modelspace()

coordinates = []

for entity in msp:
    if entity.dxftype() == 'LWPOLYLINE':
        points = entity.get_points()  # Returns (x, y, [start_width, end_width, bulge])
        for x, y, *_ in points:
            coordinates.append((x, y))

    elif entity.dxftype() == 'POLYLINE':
        for vertex in entity.vertices:
            x, y = vertex.dxf.location.x, vertex.dxf.location.y
            coordinates.append((x, y))
coordinates_mm = [(x / 1000, y / 1000) for x, y in coordinates]
for pt in coordinates_mm:
    print(pt)

(22.622435000000003, 6.976248)
(22.687435, 6.976248)
(22.687435, 7.093748)
(22.622435000000003, 7.093748)
(22.622435000000003, 7.115247999999999)
(22.687435, 7.115247999999999)
(22.687435, 7.232748)
(22.622435000000003, 7.232748)
(22.353435, 6.976248)
(22.418435000000002, 6.976248)
(22.418435000000002, 7.093748)
(22.353435, 7.093748)
(22.353435, 7.115247999999999)
(22.418435000000002, 7.115247999999999)
(22.418435000000002, 7.232748)
(22.353435, 7.232748)
(19.053937, -26.389567)
(19.143931000000002, -26.479609)
(25.901949000000002, -26.479563)
(25.991949, -26.389563)
(25.991949, -20.937562999999997)
(25.901949000000002, -20.847562999999997)
(22.672079, -18.982553)
(22.671118999999997, -18.982553)
(22.671118999999997, 6.777116)
(22.806656999999998, 6.817066)
(22.841973000000003, 6.935299)
(23.409063999999997, 6.935299)
(23.915472, 6.729566)
(24.875472000000002, 6.729566)
(24.875472000000002, 7.479566)
(23.915472, 7.479566)
(23.409165, 7.273918)
(22.841973000000003, 7.273918)
(22.8066569

In [73]:
# Multi-axis, multiple points (Drawing outline)
axes = [0,1,-1]
sp.MultiPointM(
    hc,
    sp.MotionFlags.ACSC_AMF_VELOCITY,
    axes,
    1,
    failure_check=True
)

sp.ExtAddPointM(hc, axes, (0,0), 25)
for i in range(len(coordinates_mm)):
    try:
        sp.ExtAddPointM(hc, axes, coordinates_mm[i], 25)
    except Exception as e:
        print(e)
sp.ExtAddPointM(hc, axes, (0,0), 25)
sp.EndSequenceM(hc, axes)

In [53]:
sp.KillAll(hc)

In [75]:
# Disable all
sp.DisableAll(hc, sp.SYNCHRONOUS, True)

In [76]:
# Close connection
sp.CloseComm(hc)